In [22]:
K = GF(17)
FF.<t> = FunctionField(K)
P.<x,y> = ProjectiveSpace(FF,1)
DS1 = DynamicalSystem([(1/t)*x^2+y^2 ,y^2])
DS1.normalize_coordinates()
DS1

Dynamical System of Projective Space of dimension 1 over Rational function field in t over Finite Field of size 17
  Defn: Defined on coordinates by sending (x : y) to
        (x^2 + t*y^2 : t*y^2)

In [38]:
# %load is_prime_of_good_reduction.py
# Function to check if "prime" is a prime of good reduction
def is_prime_of_good_reduction(self,prime):
    DS = copy(self)
    degree = DS.degree_sequence(1)[0]
    try:
        new_DS = mod(DS,prime)
    except:
        # the mod fucntion modDS throws exception only if
        # the polynomials of the DynamicalSystem doesn't have the same degree which
        # means that this is a prime of bad reduction
        return False
    # check if the degree of the DynamicalSystem changed.
    # yes -> prime of bad reduction
    # no -> prime of good reduction
    if new_DS.degree_sequence(1)[0] != degree :
        return False
    return True

In [42]:
def all_possible_global_periods(self,primes,e  ):
    DS = copy(self)
    base = DS.base_ring()
    CF = base.constant_field() 
    D = DS.domain()
    # check if the dimension is equal to 1
    if DS.domain().dimension() != 1 :
        raise NotImplementedError("The dimension must be 1")
    # check if the field is function field over Finite Field 
    if not isinstance(DS.base_ring(), sage.rings.function_field.function_field.RationalFunctionField_global) or CF.order() == Infinity : 
        raise TypeError("The field must be function field over finite Field ")
    all_possible_global_periods_list = []
    temp_g = 0
    for prime in primes : 
        # check if the prime is a prime of good reduction
        if is_prime_of_good_reduction(DS,prime) :
            new_DS = mod(DS,prime)
            all_periodic_points = new_DS.all_periodic_points()
            for periodic_point in all_periodic_points:
                # check if this isn't the infinity point
                if periodic_point != D(1 , 0) : 
                    temp_g += 1 # just to know what is the first list
                    period = new_DS.orbit_structure(periodic_point)[1] # period of the periodic point
                    multiplier_ = new_DS.multiplier(periodic_point,period) 
                    sub_list = []
                    # first add the period of the point
                    sub_list.append(period)
                    # first add the period of ( 1:0 ) 
                    sub_list.append(1)
                    if multiplier_[0][0] != 0 : 
                        # calculate the multiplicative order of the multiplier
                        a = CF(multiplier_[0][0])
                        m_o = a.multiplicative_order() 
                        #  Compute mrVp^e for the cycle (period)
                        for temp_e in range(0,e+1) : 
                            sub_list.append(period * m_o * (prime**temp_e) )
                    if temp_g == 1 :
                        # just to avoid making intersection
                        # between the first possible periods list and the empty list (the start of the list )
                        all_possible_global_periods_list = list(sub_list)
                    else :
                        # find the intersection between the list of the mrp^e (Proposition 2 in the article of Hutz algorithm )
                        # and the main list possible global periods list
                        all_possible_global_periods_list = list ( set(all_possible_global_periods_list) & set(sub_list))          
    return all_possible_global_periods_list            

In [43]:
# %load mod_dynamical_system_prime.py
# Function the returns the dynamical system mod p ( ds mod p )
def mod(self,p):
    DS = copy(self)
    DS.normalize_coordinates()  # normalize coordinates of dynamical system
    df_p_all = DS.defining_polynomials()
    coefficients = []
    for i in range(0,len(df_p_all)):
        coefficients.append(df_p_all[i].coefficients())
    monomials = []
    for i in range(0,len(df_p_all)):
        monomials.append(df_p_all[i].monomials())
    F = DS.base_ring() # obtain base_ring of dynamical system
    O = F.maximal_order()
    p = O.ideal(p).place()
    k, fr_k, to_k = p.residue_field()
    # reduce each coefficient of each polynomial using the function to_k :
    reduced_coefficients = []
    for i in range(0,len(coefficients)):
        reduced_coefficients.append([to_k(c) for c in coefficients[i]])
    # change the ring of each monomial to the residue field :
    new_monomials = []
    for i in range(0,len(monomials)):
        new_monomials.append([m.change_ring(k) for m in monomials[i]])
    new_pol = []
    polynom = 0
    for i in range(0,len(reduced_coefficients)):
        for j in range(0,len(reduced_coefficients[i])):
            polynom += reduced_coefficients[i][j] * new_monomials[i][j]
        new_pol.append(polynom)
        polynom = 0
    # construct a new dynamical system from the polynomials
    new_DS = DynamicalSystem(new_pol)
    return new_DS

In [44]:
lst = []
for prime in FF.places_finite(1):
    lst.append(prime.local_uniformizer())
print(lst)
all_possible_global_periods(DS1,lst,1)

[t, t + 1, t + 2, t + 3, t + 4, t + 5, t + 6, t + 7, t + 8, t + 9, t + 10, t + 11, t + 12, t + 13, t + 14, t + 15, t + 16]


[1]